# Prompt Service

In [1]:
!pip install fastapi uvicorn nest-asyncio

In [2]:
!pip install --upgrade pyngrok

In [3]:
!pip install openai

In [4]:
from pyngrok import ngrok
ngrok.set_auth_token("2lkokgaV9yas4LTJOzF2N73aROF_6iFCShmMYooSUHgM1eRrQ")

In [5]:
!

In [6]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import json
from openai import OpenAI

In [7]:
app = FastAPI()

@app.post("/getNearbyConsumables")
async def root(data: dict):
  address = data.get("address")
  if address is None:
    return {"error": "Address not provided"}
  else:
    results = process_data(address)
    print(results)
  return {"result": results}

In [8]:
def get_nearby_places(location):
    client = OpenAI(
        api_key="",  # this is also the default, it can be omitted
    )
    # final_prompt = f"Provide a list of nearby restaurants, bookstores, and other interesting places to visit near {location} in JSON format. Format the response as a JSON object with keys 'restaurants', 'bookstores', and 'places'. Don't worry about providing real-time information. Just tell me what you know. Also, please make your entire response in json format without any context."
    final_prompt = f"Provide a JSON object listing nearby restaurants, bookstores, and other interesting places " + \
                  f"to visit near {location}. The object should contain three keys: 'restaurants', 'bookstores', " + \
                  f"and 'other places'. Populate each key with a list of examples based on your knowledge. " + \
                  f"Do not include any additional text or explanations—just return the JSON object.\n\n" + \
                  f"To clarify, please do not include the word 'json' before the json object. I really do JUST want the json object."
    for _ in range(5):
        response = client.chat.completions.create(
                  model="gpt-4o",
                  messages=[
                      {"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": final_prompt}
                ]
              )
        content = response.choices[0].message.content
        try:
            nearby_places = json.loads(content)
            return nearby_places
        except json.JSONDecodeError:
            print("Response content is not valid JSON.")
            continue

In [9]:
def process_data(value):
  # prompt = f"I am at : {value}. Please provide me the best restaurants and clothing store around me."
  # print(prompt)
  results = get_nearby_places(value)
  return results # example processing: double the value

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [5340]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://febe-34-81-214-97.ngrok-free.app
INFO:     3.222.251.165:0 - "GET / HTTP/1.1" 404 Not Found
{'restaurants': [{'name': "Carmine's Italian Restaurant", 'address': '200 W 44th St, New York, NY 10036', 'cuisine': 'Italian'}, {'name': "Junior's Restaurant & Bakery", 'address': '1515 Broadway, New York, NY 10036', 'cuisine': 'American'}, {'name': 'The Capital Grille', 'address': '120 W 51st St, New York, NY 10020', 'cuisine': 'Steakhouse'}], 'bookstores': [{'name': 'The Drama Book Shop', 'address': '266 W 39th St, New York, NY 10018'}, {'name': 'bookoff', 'address': '49 W 45th St, New York, NY 10036'}, {'name': 'Kinokuniya New York', 'address': '1073 6th Ave, New York, NY 10018'}], 'other places': [{'name': "Ripley's Believe It or Not! Times Square", 'address': '234 W 42nd St, New York, NY 10036', 'type': 'Museum'}, {'name': 'Madame Tussauds New York', 'address': '234 W 42nd St, New York, NY 10036', 'type': 'Museum'}, {'name': 'Bryant Park', 'address': 'New York, NY 10018